# Training the "small" model

The code above will load training data and train the so-called "small" model.

Out of a prescribed number (given by the parameter `seq_len`) of last orders of every user, only the order date and the bill amount is taken into the account by this model. The model ignores every other features of orders.

In [14]:
from pndapetzim.data import load_dataset
from pndapetzim.data import ORDER_FILE_NAME
from pndapetzim.data import LABEL_FILE_NAME

seq_len = 20
returning_weight = 4.0

ds = load_dataset(
    '../data/' + ORDER_FILE_NAME,
    '../data/' + LABEL_FILE_NAME,
    seq_len,
    returning_weight,
).cache().shuffle(100)

In [15]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import AUC
from tensorflow.keras.metrics import Recall
from tensorflow.keras.optimizers import Adam

from pndapetzim.models import build_amount_date_model

loss = BinaryCrossentropy()
optimiser = Adam(learning_rate=0.01)

model = build_amount_date_model(seq_len, 10)
metrics = ['accuracy', Recall(), AUC()]
model.compile(loss=loss, optimizer=optimiser, metrics=metrics)

In [ ]:
batch_size = 128
model.fit(ds.batch(batch_size).prefetch(10), epochs=5)

Epoch 1/5
    685/Unknown - 172s 248ms/step - loss: 0.9666 - accuracy: 0.6977 - recall_5: 0.6899 - auc_5: 0.7584